# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [9]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*.csv'))
    #print(file_path_list)

/home/workspace


In [10]:
file_path_list[:5]

['/home/workspace/event_data/2018-11-30-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-11-events.csv']

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [11]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [12]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [13]:
from cassandra.cluster import Cluster
cluster=Cluster()

session = cluster.connect()

#### Create Keyspace

In [14]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project
    WITH REPLICATION=
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [15]:
session.set_keyspace('project')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
SELECT
    artist_name,
    song_title,
    song_length
FROM 
  SONGS_ITEMSETTION
WHERE 
    seetion_id = 338   
AND 
    item = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
SELECT

   artist_name,
   song_title,
   first_name,
   last_name
   
   
FROM 
  songs_playlist
WHERE 
     userid= 10
AND 
     sesstion_id= 182
ORDER BY
     item DESC
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

SELECT

   userid,
   first_name,
   last_name
   
FROM 
  SONGS_USERS
WHERE 
     song_title= 'All Hands Against His Own'   



In [16]:
##This statement creates the SONGS_ITEMSETTION with a primary key ( seetion_id, item ) to answer the question 1
##As the primary key has two components, the first component is considered a partition key "seetion_id"
##and the second component becomes the cluster key "item"
## we couldnot appling  the filters in the question without using partion keys
##Clustering columns order data within a partition
query1 = "CREATE TABLE IF NOT EXISTS SONGS_ITEMSETTION"
query1 += '''(

               seetion_id int,
               item int,
               artist_name text,
               song_length float,
               song_title text,
          PRIMARY KEY(seetion_id,item)     
    
    )'''


try:
    session.execute(query1)
except Exception as e:
    print(e)

In [17]:
## another code more readable by using panda 
## Load the data with INSERT statement for SONGS_ITEMSETTION table
df = pd.read_csv("event_datafile_new.csv")
query1 = "INSERT INTO SONGS_ITEMSETTION (seetion_id,item,artist_name,song_length,song_title)"
query1 =query1 + "VALUES(%s, %s, %s, %s, %s)"
for index, row in df.iterrows():
    
    try:
        session.execute(query1, (row.sessionId,
                                 row.itemInSession,
                                 row.artist,
                                 row.length,
                                 row.song)
                       )
    except Exception as e:
        print(e)
        

In [18]:
## select statment to answer question 1

query1= """
SELECT
    artist_name,
    song_title,
    song_length
FROM 
  SONGS_ITEMSETTION
WHERE 
    seetion_id = 338   
AND 
    item = 4
"""
try:
    records = session.execute(query1)
except Exception as e:
    print(e)
    
for row in records:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


In [19]:
##This statement creates the songs_playlist with a primary key ((session_id,userid),item ) to answer the question 2
##As the primary key has two components, the first component is considered a partition key "session_id,userid"
##and the second component becomes the cluster key "item"
## we couldnot appling  the filters in the question without using partion keys
##Clustering columns order data within a partition
query2 = "CREATE TABLE IF NOT EXISTS songs_playlist"
query2 += '''(

               sesstion_id int,
               userid int,
               item int,
               artist_name text,
               song_title text,
               first_name text,
               last_name text,
          PRIMARY KEY((sesstion_id,userid),item)   
    
    )'''
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [20]:
## Load the data with INSERT statement for songs_playlist table
df = pd.read_csv("event_datafile_new.csv")
query2 = "INSERT INTO songs_playlist (sesstion_id,userid,item,artist_name,song_title,first_name,last_name)"
query2 =query2 + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
for index, row in df.iterrows():
    
    try:
        session.execute(query2, (row.sessionId,
                                 row.userId,
                                 row.itemInSession,
                                 row.artist,
                                 row.song,
                                 row.firstName,
                                 row.lastName)
                                
                       )
    except Exception as e:
        print(e)
        


In [21]:
### select statment to answer question 2

query2= """
SELECT

   artist_name,
   song_title,
   first_name,
   last_name
   
   
FROM 
  songs_playlist
WHERE 
     userid= 10
AND 
     sesstion_id= 182
ORDER BY
     item DESC


"""
try:
    records = session.execute(query2)
except Exception as e:
    print(e)
    
for row in records:
    print(row)

Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')


In [22]:
##This statement creates the SONGS_USERS with a primary key (song_title,userid to answer the question 3
##As the primary key has two components, the first component is considered a partition key "song_title"
##and the second component becomes the cluster key "userid"
## we couldnot appling  the filters in the question without using partion keys
##Clustering columns order data within a partition
query3 = "CREATE TABLE IF NOT EXISTS SONGS_USERS"
query3 += '''(

               song_title text,
               userid int,
               first_name text,
               last_name text,
          PRIMARY KEY(song_title,userid)     
    
    )'''
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [23]:
## Load the data with INSERT statement for SONGS_USERS table

df = pd.read_csv("event_datafile_new.csv")
query3 = "INSERT INTO SONGS_USERS (song_title,userid,first_name,last_name)"
query3 =query3 + "VALUES(%s, %s, %s, %s)"
for index, row in df.iterrows():
    
    try:
        session.execute(query3, (row.song,
                                 row.userId,
                                 row.firstName,
                                 row.lastName)
                       )
    except Exception as e:
        print(e)
        

In [25]:
## select statment to answer question 3

query3= """
SELECT

    userid,
   first_name,
   last_name
   
FROM 
  SONGS_USERS
WHERE 
     song_title= 'All Hands Against His Own'   


"""
try:
    records = session.execute(query3)
except Exception as e:
    print(e)
    
for row in records:
    print(row)

Row(userid=29, first_name='Jacqueline', last_name='Lynch')
Row(userid=80, first_name='Tegan', last_name='Levine')
Row(userid=95, first_name='Sara', last_name='Johnson')


In [26]:
##drop tables in KEYSPACE

query = "drop table IF EXISTS SONGS_ITEMSETTION"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table IF EXISTS  songs_playlist"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table IF EXISTS  SONGS_USERS"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [56]:
session.shutdown()
cluster.shutdown()